# API In-Class Assignment
---
Jan. 14, 2026 <br>
## Team: Clouds

# Dependencies

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Data Access

In [2]:
# Define API URL
url = "https://waterdata.usgs.gov/nwis/uv?cb_all_=on&cb_00060=on&cb_00065=on&format=gif_stats&site_no=12451000&legacy=1&period=&begin_date=2025-12-11&end_date=2025-12-12"

# Download data
response = requests.get(url)
response.raise_for_status()  # Raise an error if the request failed

# localFile = 'pbdbDiversity.csv'

# # Save the response to a local file
# with open(localFile, "w", encoding="utf-8") as f:
#     f.write(response.text)